In [ ]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

# TOTAL FACTURAS 

In [19]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta , datetime

weeks = 52

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] #account_analytic_id

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
line_df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
line_df = pd.DataFrame(records).fillna(0)

# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

#df["company_id"] = df["company_id"].astype(str)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["product_id", "invoice_id","create_date"])



invoice_fields = ["id", "store_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids],
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# Separar store_id

invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)



df_merged = line_df.merge(invoice_df[["id", "store_name"]], left_on="invoice_id_num", right_on="id", how="left")
df_merged = df_merged.drop(columns=["id_x","id_y","invoice_id_num","product_id_num"])

df_merged['origin'] = df_merged['origin'].astype('string')


df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas lines account: {len(line_df)}")


#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos

Facturas encontradas: 51109
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 60000 registros...
Descargados 80000 registros...
Descargados 100000 registros...
Descargados 120000 registros...
Descargados 140000 registros...
Descargados 160000 registros...
Descargados 180000 registros...
Descargados 199320 registros...
Total de líneas descargadas lines account: 199320


In [ ]:
print(f"Fecha primera factura",df_merged["date_invoice"].min())
print(f"Fecha ultima factura",df_merged["date_invoice"].max())

In [ ]:
df_merged.sample(10)

# COSTO DEL PRODUCTO

In [ ]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [12]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [18]:
import xmlrpc.client
from datetime import date
import pandas as pd
import warnings

# Ignorar la advertencia de `SettingWithCopyWarning` temporalmente
# Esta es una buena práctica para evitar que el código se detenga
# ===============================
# 1. Conexión con Odoo
# ===============================

# Variables de conexión
ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Leer ventas perdidas
# ===============================

# Definir la fecha de inicio para la consulta (desde 2024-12-01)
FECHA_INICIO = "2024-12-01"

# Buscar los IDs de las ventas perdidas
lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["almacenamiento_tipo", "=", "agotado"]]]
)

# Leer los datos de las ventas perdidas usando los IDs
lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

# Convertir la lista de diccionarios a un DataFrame de Pandas
df = pd.DataFrame(lost_sales_data)

# ===============================
# 3. Normalizar columnas con tuplas (many2one)
# ===============================

# Usar el método .str para extraer el primer y segundo elemento de las tuplas de forma eficiente
for col in ["store_id", "cliente_id"]:
    # Crea una nueva columna para el nombre (e.g., store_name)
    df[col.replace('_id', '_name')] = df[col].apply(lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None)
    # Crea una nueva columna para el ID (e.g., store_id_num)
    df[col + "_num"] = df[col].apply(lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None)

# La columna 'product_ref' ya es una cadena y no necesita normalización
# Las columnas product_id, warehouse_id y warehouse_name no se incluyeron en el READ, así que no es necesario procesarlas

# ===============================
# 4. Limpiar el DataFrame
# ===============================

# Eliminar las columnas que no son necesarias para el análisis final
# La columna 'id' se puede eliminar ya que no es relevante
# Ahora que extrajimos el nombre, podemos eliminar las columnas originales de tuplas
df = df.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# Renombrar las columnas para mayor claridad
#df = df.rename(columns={
 #   "product_ref": "referencia_producto",
  #  "store_name": "nombre_tienda",
   # "cliente_name": "nombre_cliente",
#})



# ===============================
# 5. Exportar a Parquet
# ===============================
try:
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="pyarrow", compression="snappy")
    print("✅ Exportación exitosa con pyarrow")
except ImportError as e:
    print("⚠️ Módulo pyarrow no encontrado, probando con fastparquet:", e)
    try:
        df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="fastparquet", compression="snappy")
        print("✅ Exportación exitosa con fastparquet")
    except ImportError as e:
        print("❌ Módulo fastparquet tampoco encontrado. Por favor, instala uno de los dos con 'pip install pyarrow' o 'pip install fastparquet'.")

print("\n")
print(f"El proceso ha terminado. Se extrajeron {len(df)} registros de ventas perdidas.")
print("Las columnas del DataFrame final son:", list(df.columns))


    ##TARDA MAS O MENOS 1 MINUTOS TODO 2025


✅ Exportación exitosa con pyarrow


El proceso ha terminado. Se extrajeron 57700 registros de ventas perdidas.
Las columnas del DataFrame final son: ['cantidad_reservada', 'product_ref', 'cantidad', 'almacenamiento_tipo', 'fecha', 'cantidad_existencia', 'origen', 'store_name', 'store_id_num', 'cliente_name', 'cliente_id_num']


# MOTOR BUSQUEDA

In [28]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "mrp.routing", "fields_get", 
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")



active                    | boolean | Active
code                      | char | Code
company_id                | many2one | Company
create_uid                | many2one | Created by
create_date               | datetime | Created on
note                      | text | Description
display_name              | char | Display Name
employees_ids             | many2many | Empleados habilitados
id                        | integer | Id
__last_update             | datetime | Last Modified on
write_uid                 | many2one | Last Updated by
write_date                | datetime | Last Updated on
name                      | char | Name
location_id               | many2one | Production Location
workcenter_lines          | one2many | Work Centers


In [ ]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")

#aprox2min

In [ ]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-09"]
r.head(30)